С удовольствием! Вот подробный разбор **Модуля 1: Введение в Nginx**, как будто я веду занятие у студентов.

---

# 📘 Модуль 1: Введение в Nginx

---

## 🎯 Цель модуля:

* Понять, что такое Nginx и зачем он нужен.
* Установить Nginx.
* Разобраться со структурой конфигурации.
* Поднять свой первый локальный сайт.

---

## 🧠 Теория

### 🔹 Что такое веб-сервер?

Веб-сервер — это программа, которая принимает HTTP-запросы от клиентов (например, браузеров) и отдает им HTTP-ответы (например, HTML-страницы).

Есть два вида:

* **Static Web Server** — просто отдает HTML, CSS, JS.
* **Dynamic Web Server** — работает с backend-языками (например, через проксирование).

### 🔹 Что такое Nginx?

**Nginx (произносится "Engine-X")** — это мощный, легковесный веб-сервер с открытым исходным кодом. Его ключевые особенности:

* Высокая производительность
* Низкое потребление ресурсов
* Поддержка реверс-прокси
* Балансировка нагрузки
* Кеширование
* SSL/TLS
* Расширяемость

Он работает по **event-driven** архитектуре — не создает по одному потоку на каждый запрос (как Apache), а обрабатывает их неблокирующе.

---

## 🔧 Установка Nginx

### 📦 Способ 1: Установка через пакетный менеджер (Linux)

**Ubuntu/Debian:**

```bash
sudo apt update
sudo apt install nginx
```

**CentOS/RHEL:**

```bash
sudo yum install epel-release
sudo yum install nginx
```

**Запуск и проверка:**

```bash
sudo systemctl start nginx
sudo systemctl enable nginx
```

Затем откройте в браузере:
📎 [http://localhost](http://localhost) — и вы должны увидеть "Welcome to nginx!"

---

### 📦 Способ 2: Установка через Docker

Если у вас установлен Docker:

```bash
docker run --name nginx-test -p 8080:80 -d nginx
```

Перейдите в браузере по адресу [http://localhost:8080](http://localhost:8080)
Nginx будет работать в контейнере.

---

### 📁 Структура директорий Nginx

| Файл / Папка            | Назначение                                 |
| ----------------------- | ------------------------------------------ |
| `/etc/nginx/nginx.conf` | Главный конфигурационный файл              |
| `/etc/nginx/conf.d/`    | Дополнительные конфигурации (обычно .conf) |
| `/var/www/html/`        | Директория с HTML-файлами по умолчанию     |
| `/var/log/nginx/`       | Логи сервера                               |

---

### 📄 Разбор `nginx.conf`

Пример базовой структуры:

```nginx
user www-data;
worker_processes auto;
error_log /var/log/nginx/error.log;
pid /run/nginx.pid;

events {
    worker_connections 1024;
}

http {
    include       mime.types;
    default_type  application/octet-stream;

    access_log  /var/log/nginx/access.log;

    sendfile        on;
    keepalive_timeout  65;

    include /etc/nginx/conf.d/*.conf;
}
```

Здесь:

* `user`, `worker_processes` — настройки ОС
* `events` — настройка соединений
* `http` — главный блок для веб-настроек
* В `http` можно указывать `server {}` блоки (виртуальные хосты)

---

## 🧪 Практика

### 🔧 Установка + проверка

1. Установите Nginx (любой способ).
2. Откройте браузер и перейдите на `http://localhost`
3. Найдите файл `nginx.conf` и откройте его.
4. Найдите строку `include /etc/nginx/conf.d/*.conf;` — это значит, что конфигурация может быть разбита на отдельные файлы.
5. Перейдите в `/etc/nginx/conf.d/` и создайте новый файл `test.conf`:

```nginx
server {
    listen 8080;
    server_name localhost;

    location / {
        return 200 "Hello from custom config!\n";
    }
}
```

6. Перезапустите сервер:

```bash
sudo nginx -t && sudo systemctl reload nginx
```

7. Проверьте в браузере: [http://localhost:8080](http://localhost:8080)

---

## 📝 Домашнее задание

1. Установить Nginx вручную и через Docker.
2. Найти файл `nginx.conf` и описать, что делает каждая секция.
3. Создать кастомный `server`-блок, который слушает порт 8081 и отдает простое сообщение.
4. *(Дополнительно)* Настроить HTML-страницу в `/var/www/custom` и раздать её через свой server block.

---

## 📌 Вопросы для самопроверки

* Чем отличается Apache от Nginx?
* Что такое worker process?
* Где искать конфигурации серверов?
* Как проверить конфигурацию перед перезапуском?
* Что произойдёт, если запустить Nginx в Docker и на хосте одновременно?

---

Хочешь, чтобы я перешел ко **второму модулю**, или подготовил тебе **шаблон лабораторной работы по первому модулю**?


Конечно! Давайте разберем **Модуль 2: Основы конфигурации Nginx**, как будто это полноценное занятие для студентов 👩‍🏫

---

# 📘 Модуль 2: Основы конфигурации Nginx

---

## 🎯 Цель модуля:

* Изучить структуру и синтаксис конфигурационных файлов Nginx
* Понять, что такое **директивы**, **блоки**, **контексты**
* Создать **виртуальные хосты (server blocks)** для нескольких сайтов
* Разобраться с **логами** (`access.log`, `error.log`)

---

## 🧠 Теория

### 🔹 Что такое конфигурационный файл Nginx?

Это текстовый файл, где описано поведение сервера:

* Какой порт слушать
* Где искать файлы сайта
* Как проксировать запросы
* Как логировать ошибки и запросы
* Как обрабатывать разные URL

Главный файл конфигурации — `/etc/nginx/nginx.conf`
Но обычно конфигурация разбивается на:

* `/etc/nginx/nginx.conf` — ядро
* `/etc/nginx/conf.d/*.conf` — сайты
* `/etc/nginx/sites-available/` и `/sites-enabled/` — альтернативная схема на Ubuntu

---

### 🔹 Основные термины

#### ✅ **Директива**

Это одна строка, задающая конкретное поведение:

```nginx
listen 80;
root /var/www/mysite;
```

#### ✅ **Блок**

Группа директив, заключённая в `{ }`:

```nginx
server {
    listen 80;
    server_name example.com;

    location / {
        root /var/www/example;
    }
}
```

#### ✅ **Контексты**

Nginx имеет **иерархическую структуру** конфигурации:

* **main** — корень (в `nginx.conf`)
* **http** — всё, связанное с веб-протоколом
* **server** — виртуальный хост
* **location** — обработка путей внутри сайта

---

### 🔹 Пример server block (виртуальный хост)

```nginx
server {
    listen 80;
    server_name example.com www.example.com;

    root /var/www/example;
    index index.html;

    access_log /var/log/nginx/example_access.log;
    error_log /var/log/nginx/example_error.log;

    location / {
        try_files $uri $uri/ =404;
    }
}
```

**Объяснение:**

* `listen 80;` — слушает порт 80
* `server_name` — доменные имена, которые будут обрабатываться
* `root` — путь к HTML-файлам
* `index` — файл по умолчанию
* `location /` — обработка всех URL
* `try_files` — попытка найти файл, иначе ошибка 404

---

## 🔧 Практика

### 🔨 1. Создание кастомного server block

1. Создайте директорию для сайта:

```bash
sudo mkdir -p /var/www/site1
echo "<h1>Hello from site1</h1>" | sudo tee /var/www/site1/index.html
```

2. Создайте конфигурацию:

```bash
sudo nano /etc/nginx/conf.d/site1.conf
```

```nginx
server {
    listen 8081;
    server_name site1.local;

    root /var/www/site1;
    index index.html;

    location / {
        try_files $uri $uri/ =404;
    }
}
```

3. Добавьте в `/etc/hosts`:

```
127.0.0.1 site1.local
```

4. Перезапустите Nginx:

```bash
sudo nginx -t
sudo systemctl reload nginx
```

5. Откройте в браузере:
   📎 [http://site1.local:8081](http://site1.local:8081)

---

### 🔍 2. Работа с логами

По умолчанию:

* `access.log` показывает все HTTP-запросы (GET, POST и др.)
* `error.log` — ошибки (например, 404, 500)

Пример записи в access.log:

```
127.0.0.1 - - [01/Aug/2025:15:00:00 +0300] "GET / HTTP/1.1" 200 612 "-" "Mozilla/5.0"
```

Можно задать индивидуальные логи для каждого сайта в `server`-блоке:

```nginx
access_log /var/log/nginx/site1_access.log;
error_log /var/log/nginx/site1_error.log warn;
```

---

## 📝 Домашнее задание

1. Создать **два сайта** (`site1.local` и `site2.local`), каждый на своём порту.
2. Настроить для них **индивидуальные директории** и **логи**.
3. Проверить работу каждого сайта через браузер и посмотреть логи командой:

```bash
tail -f /var/log/nginx/site1_access.log
```

---

## 📌 Вопросы для самопроверки

* Что такое директива? Что такое блок?
* Как работает `server_name`?
* Что делает `try_files $uri $uri/ =404;`?
* Где можно задать индивидуальный лог?
* Как Nginx понимает, на какой конфигурации "ответить"?

---

## 💡 Советы

* Всегда проверяй конфигурацию перед перезапуском:

```bash
sudo nginx -t
```

* Не редактируй `nginx.conf` напрямую, создавай конфиги в `conf.d/`
* При ошибке 403 — скорее всего, неправильно указан `root` или права на директорию

---

Если хочешь, я могу:

* Перейти к **Модулю 3**
* Подготовить **лабораторную работу по этому модулю**
* Сделать **шпаргалку** по `server` и `location` блокам

Что бы ты хотела дальше?


Отлично! Продолжаем. Ниже — подробное объяснение **Модуля 3: Обработка статических файлов в Nginx**, как будто я веду занятие в группе.

---

# 📘 Модуль 3: Обработка статических файлов в Nginx

---

## 🎯 Цель модуля:

* Научиться отдавать HTML, CSS, JS, изображения и другие статические файлы через Nginx.
* Понять, как работает `root` и `alias`.
* Настроить кеширование и корректные MIME-типы.
* Разрешить или запретить просмотр содержимого директорий.

---

## 🧠 Теория

### 🔹 Что такое «статический файл»?

Это файл, содержимое которого не изменяется при каждом запросе:

* HTML, CSS, JavaScript
* Изображения (PNG, JPEG, SVG)
* Шрифты, PDF, MP4 и т.д.

Nginx идеально подходит для раздачи таких файлов, т.к. делает это быстро и эффективно.

---

### 🔹 Ключевые директивы

#### ✅ `root`

Указывает корень файловой системы для URL:

```nginx
location /images/ {
    root /var/www/site;
}
```

`/images/cat.png` → ищется как: `/var/www/site/images/cat.png`

#### ✅ `alias`

Позволяет указать другой путь (без добавления части URI):

```nginx
location /images/ {
    alias /var/www/static/;
}
```

`/images/cat.png` → ищется как: `/var/www/static/cat.png` (без `/images`)

> 📌 **root** добавляет URI к пути, **alias** — заменяет путь полностью.

---

### 🔹 MIME-типы

МIME-типы — это метки, указывающие браузеру, что за файл получен.
Nginx подгружает их из файла `mime.types`, который подключается в `http`:

```nginx
include mime.types;
default_type application/octet-stream;
```

Примеры:

* `.html` → `text/html`
* `.css` → `text/css`
* `.png` → `image/png`

---

### 🔹 Индексация директорий

Если в папке нет `index.html`, можно разрешить показ списка файлов:

```nginx
location /files/ {
    root /var/www/site;
    autoindex on;
}
```

Это может быть полезно в админке или закрытых файловых хранилищах.

---

### 🔹 Кеширование

Чтобы браузер не запрашивал файл каждый раз:

```nginx
location ~* \.(jpg|png|gif|ico|css|js)$ {
    expires 7d;
    add_header Cache-Control "public";
}
```

* `expires 7d` — скажи браузеру кэшировать на 7 дней.
* `add_header` — явно добавляет заголовок.

---

## 🔧 Практика

### 🔨 1. Раздача HTML и изображений

1. Создайте директорию:

```bash
sudo mkdir -p /var/www/static-site/images
```

2. Создайте HTML-файл:

```html
<!-- /var/www/static-site/index.html -->
<!DOCTYPE html>
<html>
<head><title>My Site</title></head>
<body>
  <h1>Hello from static site</h1>
  <img src="/images/cat.png" alt="cat">
</body>
</html>
```

3. Добавьте изображение (`cat.png`) в `/images`

4. Настройте конфигурацию:

```nginx
server {
    listen 8082;
    server_name static.local;

    root /var/www/static-site;
    index index.html;

    location / {
        try_files $uri $uri/ =404;
    }

    location /images/ {
        expires 30d;
    }
}
```

5. Проверь:

```bash
sudo nginx -t && sudo systemctl reload nginx
```

6. Перейди в браузер:
   📎 [http://static.local:8082](http://static.local:8082)

---

### 🔍 2. Использование `alias`

Допустим, файлы лежат в `/var/files`, а ты хочешь отдавать их по `/downloads/`:

```nginx
location /downloads/ {
    alias /var/files/;
    autoindex on;
}
```

Теперь:
📎 [http://static.local:8082/downloads/file.zip](http://static.local:8082/downloads/file.zip)
→ отдаст `/var/files/file.zip`

---

### 🧪 3. Пример с кешем и gzip

```nginx
location ~* \.(css|js|jpg|jpeg|png|gif|ico)$ {
    expires 7d;
    add_header Cache-Control "public";
    gzip_static on;
}
```

Для gzip:

* Убедись, что `gzip` включён в `http`:

```nginx
gzip on;
gzip_types text/css application/javascript image/svg+xml;
```

---

## 📝 Домашнее задание

1. Создай сайт, отдающий HTML + изображения + стили.
2. Настрой кеширование изображений на 7 дней.
3. Разреши индексацию директорий `/files/`
4. Добавь `alias` на отдельную директорию (`/downloads`)

---

## 📌 Вопросы для самопроверки

* В чем разница между `root` и `alias`?
* Как работает `expires`?
* Зачем нужен `mime.types`?
* Как включить показ содержимого папки?
* Где включается gzip-сжатие?

---

Хочешь, чтобы я перешел к **Модулю 4: Реверс-прокси**, или сделать паузу и ответить на вопросы?


Отлично! Продолжаем. Ниже подробный разбор **Модуля 4: Реверс-прокси и проксирование запросов в Nginx** — как на практическом занятии для студентов.

---

# 📘 Модуль 4: Реверс-прокси и проксирование запросов

---

## 🎯 Цель модуля:

* Понять, что такое **реверс-прокси** и зачем он нужен.
* Настроить Nginx как реверс-прокси к backend-приложению.
* Научиться работать с заголовками (`X-Real-IP`, `Host`).
* Обработать ошибки backend-сервиса (`502`, `504`).
* Провести первую интеграцию с backend-сервером.

---

## 🧠 Теория

### 🔹 Что такое реверс-прокси?

**Реверс-прокси (reverse proxy)** — это сервер, который принимает запросы от клиента и **пересылает** их на другой сервер (обычно backend).

🧭 Типичная схема:

```
Клиент → Nginx (reverse proxy) → Backend (Flask, Node.js, etc)
```

Nginx:

* **скрывает backend** от клиента
* **кеширует** ответы
* **сжимает** трафик
* **балансирует нагрузку**

---

### 🔹 Директива `proxy_pass`

```nginx
location /api/ {
    proxy_pass http://localhost:5000/;
}
```

Важно:

* URI **принимается как есть**, но может быть переписан.
* Последний `/` в `proxy_pass` означает сохранение URI.
  Пример:
* Запрос: `/api/user`
* Переадресуется: `http://localhost:5000/user`

---

### 🔹 Важные заголовки при проксировании

Чтобы backend знал, откуда пришёл клиент:

```nginx
proxy_set_header Host $host;
proxy_set_header X-Real-IP $remote_addr;
proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
proxy_set_header X-Forwarded-Proto $scheme;
```

> ❗ По умолчанию Nginx может передавать "localhost" вместо реального IP клиента — поэтому эти заголовки нужны.

---

### 🔹 Прокси + ошибки

Если backend упал или не отвечает — будет `502 Bad Gateway` или `504 Gateway Timeout`. Для таких случаев можно:

```nginx
error_page 502 504 = /custom_502.html;
```

---

## 🔧 Практика

### 🔨 1. Подключение backend-приложения

Создадим простое backend-приложение (на Python + Flask):

```bash
pip install flask
```

```python
# app.py
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello from Flask!"

app.run(host='0.0.0.0', port=5000)
```

Запускаем:

```bash
python app.py
```

Теперь настроим Nginx:

```nginx
server {
    listen 8083;
    server_name backend.local;

    location / {
        proxy_pass http://127.0.0.1:5000/;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
    }
}
```

Проверь:

```bash
sudo nginx -t && sudo systemctl reload nginx
```

Теперь перейди в браузер:
📎 [http://backend.local:8083](http://backend.local:8083)
Ты должен увидеть ответ из Flask-приложения.

---

### 🧪 2. Проверка X-Real-IP

Добавь в Flask отображение IP:

```python
from flask import request

@app.route('/ip')
def get_ip():
    return request.headers.get("X-Real-IP", "no ip")
```

Открой:
📎 [http://backend.local:8083/ip](http://backend.local:8083/ip)
→ Должен быть твой IP (если проксирование работает корректно).

---

### 🔥 3. Симуляция ошибки 502

Останови Flask:

```bash
CTRL+C
```

И снова открой сайт → Nginx покажет:

```
502 Bad Gateway
```

Чтобы сделать пользовательскую страницу:

```nginx
error_page 502 =200 /502.html;

location = /502.html {
    root /var/www/errors;
}
```

Создай файл `/var/www/errors/502.html`.

---

## 📝 Домашнее задание

1. Напиши backend-приложение (на Flask, Node.js, или другим), которое возвращает JSON.
2. Проксируй запросы к нему через Nginx на `localhost:8084`
3. Настрой заголовки `X-Real-IP` и `X-Forwarded-*`
4. Настрой кастомную страницу ошибки 502
5. *(Дополнительно)* Прокси только те URL, что начинаются с `/api/`, а остальные — отдавай как статику.

---

## 📌 Вопросы для самопроверки

* Что делает `proxy_pass`?
* Как передать IP клиента backend-серверу?
* Как устроена цепочка `Client → Nginx → Backend`?
* Как обрабатываются ошибки 502, 504?
* В чем разница между `proxy_pass http://backend` и `http://backend/` (слэш в конце)?

---

Если хочешь, могу сразу перейти к **Модулю 5: Балансировка нагрузки**, либо подождать — скажи, когда готова.


Отлично, переходим к **Модулю 5: Балансировка нагрузки в Nginx** — это один из самых интересных и полезных разделов, особенно для тех, кто строит масштабируемые веб-приложения.

---

# 📘 Модуль 5: Балансировка нагрузки (Load Balancing)

---

## 🎯 Цель модуля:

* Понять, что такое балансировка нагрузки и зачем она нужна.
* Настроить Nginx как балансировщик между несколькими backend-серверами.
* Освоить разные алгоритмы балансировки.
* Обработать отказ узлов (health-check логика через `max_fails` и `fail_timeout`).
* Проверить распределение нагрузки на практике.

---

## 🧠 Теория

### 🔹 Что такое балансировка нагрузки?

Когда у нас много пользователей и один backend-сервер не справляется, мы запускаем **несколько экземпляров приложения** и **распределяем** запросы между ними.

🧭 Схема:

```
Клиенты
   ↓
[ Nginx ] ← балансировка
   ↓      ↓      ↓
 App1   App2   App3
```

Это:

* Увеличивает производительность
* Делает систему отказоустойчивой
* Позволяет горизонтально масштабироваться

---

### 🔹 Как это выглядит в Nginx

```nginx
upstream backend_cluster {
    server 127.0.0.1:5001;
    server 127.0.0.1:5002;
    server 127.0.0.1:5003;
}

server {
    listen 8085;

    location / {
        proxy_pass http://backend_cluster;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

---

## ⚙️ Алгоритмы балансировки в Nginx

### ✅ **Round Robin (по умолчанию)**

Запросы идут по очереди: 1 → 2 → 3 → 1 → 2 ...

```nginx
upstream backend {
    server 127.0.0.1:5001;
    server 127.0.0.1:5002;
}
```

---

### ✅ **Least Connections**

Сервер с **меньшим числом текущих соединений** получает запрос:

```nginx
upstream backend {
    least_conn;
    server 127.0.0.1:5001;
    server 127.0.0.1:5002;
}
```

---

### ✅ **IP Hash**

Запросы от одного и того же клиента всегда идут на один сервер (если IP не меняется):

```nginx
upstream backend {
    ip_hash;
    server 127.0.0.1:5001;
    server 127.0.0.1:5002;
}
```

---

### 🔹 Настройки отказоустойчивости

```nginx
server 127.0.0.1:5001 max_fails=3 fail_timeout=10s;
```

* `max_fails=3` — если зафиксировано 3 ошибки
* `fail_timeout=10s` — в течение 10 секунд — сервер исключается из пула

---

## 🔧 Практика

### 🔨 1. Запуск 3-х backend-серверов

Создай 3 Flask-сервера с разными портами и ответами:

```python
# app.py
from flask import Flask
import os

app = Flask(__name__)

@app.route("/")
def hello():
    return f"Hello from port {os.getenv('PORT')}!"

app.run(port=int(os.getenv("PORT")), host="0.0.0.0")
```

Запусти 3 экземпляра:

```bash
PORT=5001 python app.py
PORT=5002 python app.py
PORT=5003 python app.py
```

---

### 🔧 2. Настрой Nginx на балансировку

```nginx
upstream flask_apps {
    server 127.0.0.1:5001;
    server 127.0.0.1:5002;
    server 127.0.0.1:5003;
}

server {
    listen 8086;
    server_name lb.local;

    location / {
        proxy_pass http://flask_apps;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

Открой:
📎 [http://lb.local:8086](http://lb.local:8086)

Каждое обновление страницы — должен быть другой backend.

---

### 🔧 3. Проверка отказа сервера

1. Заверши `python app.py` на одном порту.
2. Nginx выдаст 502, пока не обновится `fail_timeout`.

Чтобы не ждать, добавь:

```nginx
server 127.0.0.1:5002 max_fails=1 fail_timeout=5s;
```

---

## 📝 Домашнее задание

1. Запусти 3 backend-сервера.
2. Настрой балансировку с алгоритмом `least_conn`.
3. Убей один сервер и проверь, как Nginx обходит его.
4. *(Дополнительно)* Настрой `ip_hash` и проверь, идёт ли клиент всегда на один backend (можно через curl с фиксированным IP).

---

## 📌 Вопросы для самопроверки

* Какой алгоритм используется по умолчанию?
* Когда стоит использовать `ip_hash`?
* Что происходит при отказе одного сервера?
* Что делают `max_fails` и `fail_timeout`?
* Почему полезно иметь несколько backend-серверов?

---

Если ты готова — могу перейти к **Модулю 6: Безопасность и SSL**, где мы настроим HTTPS и базовую защиту. Продолжим?


Отлично, переходим к **Модулю 6: Безопасность и SSL в Nginx**. Это ключевой модуль, если вы планируете размещать сайт или API в продакшене.

---

# 📘 Модуль 6: Безопасность и SSL (HTTPS) в Nginx

---

## 🎯 Цель модуля:

* Настроить защищённое соединение через HTTPS
* Получить и использовать SSL-сертификаты
* Понимать разницу между самоподписанными и официальными сертификатами
* Улучшить безопасность HTTP-заголовками
* Настроить базовую авторизацию (`auth_basic`)

---

## 🧠 Теория

### 🔹 Что такое HTTPS?

HTTPS — это **HTTP + шифрование через TLS/SSL**. Используется для:

* Защиты данных от перехвата
* Подтверждения подлинности сайта
* Обеспечения доверия пользователей (замок в браузере 🔒)

### 🔹 Что такое SSL-сертификат?

Это цифровой документ, выданный CA (сертификационным центром), который подтверждает право использовать определённый домен.

---

## 📦 Варианты сертификатов:

| Тип               | Где использовать        | Как получить                    |
| ----------------- | ----------------------- | ------------------------------- |
| **Let's Encrypt** | Продакшен               | Автоматически через `certbot`   |
| **Self-signed**   | Тестовая среда, локалка | Сгенерировать вручную `openssl` |

---

### 🔹 Пример HTTPS-конфига

```nginx
server {
    listen 443 ssl;
    server_name secure.local;

    ssl_certificate     /etc/nginx/ssl/fullchain.pem;
    ssl_certificate_key /etc/nginx/ssl/privkey.pem;

    ssl_protocols TLSv1.2 TLSv1.3;
    ssl_ciphers HIGH:!aNULL:!MD5;

    location / {
        root /var/www/html;
        index index.html;
    }
}
```

---

## 🔐 Безопасные заголовки

Добавим защиту от атак:

```nginx
add_header X-Frame-Options "SAMEORIGIN";
add_header X-XSS-Protection "1; mode=block";
add_header X-Content-Type-Options "nosniff";
```

* `X-Frame-Options`: защита от clickjacking
* `X-XSS-Protection`: активация фильтра XSS
* `X-Content-Type-Options`: запрет угадывания типа содержимого

---

## 🔑 Базовая HTTP-авторизация

```nginx
location /admin/ {
    auth_basic "Restricted Area";
    auth_basic_user_file /etc/nginx/.htpasswd;
}
```

Создай файл с пользователями:

```bash
sudo apt install apache2-utils
htpasswd -c /etc/nginx/.htpasswd admin
```

---

## 🔧 Практика

### 🔨 1. Самоподписанный сертификат (локально)

1. Создай директорию и ключи:

```bash
sudo mkdir -p /etc/nginx/ssl
openssl req -x509 -nodes -days 365 -newkey rsa:2048 \
  -keyout /etc/nginx/ssl/privkey.pem \
  -out /etc/nginx/ssl/fullchain.pem
```

Укажи:

* Country: RU
* Common Name: `secure.local`

2. Создай конфигурацию:

```nginx
server {
    listen 443 ssl;
    server_name secure.local;

    ssl_certificate     /etc/nginx/ssl/fullchain.pem;
    ssl_certificate_key /etc/nginx/ssl/privkey.pem;

    add_header X-Frame-Options "SAMEORIGIN";
    add_header X-XSS-Protection "1; mode=block";
    add_header X-Content-Type-Options "nosniff";

    location / {
        return 200 "Secure connection OK\n";
    }
}
```

3. Добавь в `/etc/hosts`:

```
127.0.0.1 secure.local
```

4. Перезапусти Nginx и открой:
   📎 [https://secure.local](https://secure.local)
   (браузер предупредит, что сертификат не доверенный — это нормально в случае self-signed)

---

### 🔐 2. Базовая авторизация

Добавим защищённый раздел:

```nginx
location /secret/ {
    auth_basic "Private";
    auth_basic_user_file /etc/nginx/.htpasswd;

    return 200 "Secret zone!";
}
```

Создаём пользователя:

```bash
htpasswd -c /etc/nginx/.htpasswd admin
```

Теперь при открытии `https://secure.local/secret/` браузер запросит логин/пароль.

---

## 📝 Домашнее задание

1. Создай HTTPS-сервер с самоподписанным сертификатом.
2. Настрой заголовки безопасности.
3. Добавь раздел `/admin/`, защищённый базовой авторизацией.
4. *(Дополнительно)* Получи реальный сертификат с помощью Let's Encrypt (`certbot`) на VPS или реальном домене.

---

## 📌 Вопросы для самопроверки

* В чем разница между самоподписанным и реальным сертификатом?
* Что делает `auth_basic`?
* Какие заголовки помогают защититься от XSS?
* Где хранятся ключи и сертификаты?
* Что произойдёт, если сертификат просрочен?

---

Хочешь, чтобы я перешёл к **Модулю 7: Продвинутая конфигурация (URL-перезапись, кеш, сжатие, лимиты)**?


Отлично! Переходим к **Модулю 7: Продвинутая конфигурация Nginx**. Этот модуль особенно важен, если вы хотите глубже понять, как оптимизировать производительность, управлять трафиком и контролировать поведение клиентов.

---

# 📘 Модуль 7: Продвинутая конфигурация в Nginx

---

## 🎯 Цель модуля:

* Научиться переписывать URL (rewrite, redirect)
* Настроить кеширование контента
* Включить сжатие gzip
* Ограничить частоту запросов и размер тела запроса
* Улучшить производительность

---

## 🔁 1. Перенаправления и переписывания (rewrite и return)

### 🔸 Постоянный редирект (301)

```nginx
server {
    listen 80;
    server_name example.com;

    return 301 https://example.com$request_uri;
}
```

### 🔸 Временный редирект (302)

```nginx
return 302 /new-path/;
```

### 🔸 Перезапись пути (rewrite)

```nginx
location /old/ {
    rewrite ^/old/(.*)$ /new/$1 permanent;
}
```

---

## 🧠 2. Кеширование статики

Кеширование экономит ресурсы сервера и ускоряет загрузку:

```nginx
location ~* \.(jpg|jpeg|png|gif|ico|css|js)$ {
    expires 30d;
    access_log off;
    add_header Cache-Control "public";
}
```

---

## 🗜 3. Сжатие контента (gzip)

Уменьшает объём передаваемых данных:

```nginx
gzip on;
gzip_types text/plain application/javascript text/css application/json;
gzip_min_length 1000;
gzip_comp_level 5;
```

> ⚠️ Не стоит включать gzip для изображений и PDF — они уже сжаты.

---

## 🚦 4. Ограничения по телу запроса

### 🔸 Ограничим размер загружаемого файла:

```nginx
client_max_body_size 10M;
```

> Ошибка 413 возникнет при превышении лимита.

---

## 🔒 5. Ограничение частоты запросов (rate limiting)

### Шаг 1: Создай зону лимита

```nginx
limit_req_zone $binary_remote_addr zone=one:10m rate=1r/s;
```

### Шаг 2: Применим лимит

```nginx
server {
    location /api/ {
        limit_req zone=one burst=5 nodelay;
    }
}
```

* `rate=1r/s`: 1 запрос в секунду
* `burst=5`: допускается 5 запросов в "порыве"
* `nodelay`: сразу обрывать, а не ждать

---

## ⚙️ 6. Кеш с proxy\_cache

Часто используется для API или бэкендов:

```nginx
proxy_cache_path /tmp/nginx_cache levels=1:2 keys_zone=my_cache:10m inactive=60m;
```

```nginx
location /api/ {
    proxy_pass http://backend:8000;

    proxy_cache my_cache;
    proxy_cache_valid 200 302 10m;
    proxy_cache_use_stale error timeout updating;
}
```

---

## 🔧 Пример полной конфигурации

```nginx
http {
    gzip on;
    gzip_types text/css application/javascript;

    limit_req_zone $binary_remote_addr zone=api_limit:10m rate=2r/s;

    proxy_cache_path /tmp/nginx_cache levels=1:2 keys_zone=my_cache:10m inactive=60m;

    server {
        listen 80;
        server_name example.com;

        client_max_body_size 5M;

        location ~* \.(css|js|jpg|png)$ {
            expires 30d;
            access_log off;
        }

        location /api/ {
            limit_req zone=api_limit burst=5;
            proxy_pass http://localhost:5000;
            proxy_cache my_cache;
            proxy_cache_valid 200 5m;
        }
    }
}
```

---

## 📝 Домашнее задание

1. Реализуй редирект с HTTP на HTTPS.
2. Настрой кеширование статики (`.css`, `.js`, `.jpg`).
3. Включи gzip и проверь `Content-Encoding: gzip` в браузере.
4. Ограничь частоту запросов к `/api/`.
5. Установи `client_max_body_size` в 2M и проверь реакцию на большие POST-запросы.

---

## 📌 Вопросы для самопроверки

* Чем отличается `rewrite` от `return`?
* Как включить gzip только для определённых типов файлов?
* Как отследить кеширование в ответе (`X-Cache-Status`, `Age`)?
* Что делает директива `client_max_body_size`?
* Как `proxy_cache_use_stale` помогает при ошибках?

---

Готов перейти к **Модулю 8: Nginx как обратный прокси, балансировщик нагрузки и WebSocket-прокси**?


Отлично! Переходим к **Модулю 8: Nginx как обратный прокси, балансировщик нагрузки и WebSocket-прокси** — одному из самых практично полезных модулей, особенно если вы работаете с микросервисами, backend-frontend архитектурой или WebSocket-приложениями.

---

# 📘 Модуль 8: Обратный прокси, балансировка нагрузки и WebSocket в Nginx

---

## 🎯 Цель модуля:

* Понять, что такое обратный прокси
* Настроить балансировку между несколькими бэкендами
* Организовать поддержку WebSocket через Nginx

---

## 🔄 1. Обратный прокси

Nginx может принимать запросы от клиентов и передавать их внутрь сети к backend-сервису:

```nginx
server {
    listen 80;
    server_name example.com;

    location / {
        proxy_pass http://localhost:5000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

### 🔹 Ключевые директивы:

* `proxy_pass`: адрес backend-сервера
* `proxy_set_header`: передаёт нужные заголовки (иначе backend не увидит IP клиента)

---

## ⚖️ 2. Балансировка нагрузки

### Пример с двумя бэкендами:

```nginx
upstream backend_cluster {
    server 127.0.0.1:5000;
    server 127.0.0.1:5001;
}

server {
    listen 80;

    location / {
        proxy_pass http://backend_cluster;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

### 🔸 Варианты стратегий балансировки:

* **по умолчанию** – *round-robin*
* `least_conn` – наименьшее количество соединений
* `ip_hash` – постоянство пользователя (по IP)

```nginx
upstream backend_cluster {
    least_conn;
    server backend1.local:8000;
    server backend2.local:8000;
}
```

---

## 🧩 3. Прокси для WebSocket

WebSocket — двусторонний протокол, который требует особой настройки:

```nginx
location /ws/ {
    proxy_pass http://localhost:3000;
    
    proxy_http_version 1.1;
    proxy_set_header Upgrade $http_upgrade;
    proxy_set_header Connection "upgrade";
    
    proxy_set_header Host $host;
    proxy_set_header X-Real-IP $remote_addr;
}
```

> ⚠️ Обратите внимание: `Upgrade` и `Connection` обязательны для WebSocket-соединения.

---

## 🛡 4. Таймауты и защита

### Увеличение таймаута:

```nginx
proxy_read_timeout 120;
```

### Ограничение доступа:

```nginx
location /admin {
    allow 192.168.1.0/24;
    deny all;
}
```

---

## ⚙️ Пример полной конфигурации

```nginx
upstream api_backend {
    server backend1:8000;
    server backend2:8000;
}

server {
    listen 80;
    server_name myapp.local;

    location / {
        proxy_pass http://frontend:3000;
    }

    location /api/ {
        proxy_pass http://api_backend;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }

    location /ws/ {
        proxy_pass http://websocket_backend:4000;

        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection "upgrade";
    }
}
```

---

## 📝 Домашнее задание

1. Настрой Nginx как прокси для backend-приложения.
2. Создай upstream с двумя backend'ами и проверь балансировку.
3. Настрой поддержку WebSocket.
4. Добавь ограничения IP-доступа к `/admin`.
5. Протестируй работу при отключении одного из backend'ов (Nginx должен продолжать обслуживать запросы).

---

## 📌 Вопросы для самопроверки

* Чем `proxy_pass` отличается от `rewrite`?
* Какие заголовки нужно обязательно проксировать?
* Какие стратегии балансировки доступны в Nginx?
* Что делает директива `ip_hash`?
* Какие настройки требуются для проксирования WebSocket?

---

Если готов, перейдём к **Модулю 9: Безопасность в Nginx** – там поговорим про HTTPS, HSTS, защиту от атак и многое другое. Готов?


Отлично! Переходим к **Модулю 9: Безопасность в Nginx** — очень важному разделу, особенно при развёртывании production-сервисов. Здесь мы научимся защищать наши сайты и API от перехвата, атак и уязвимостей.

---

# 🛡️ Модуль 9: Безопасность в Nginx

---

## 🎯 Цели модуля:

* Настроить HTTPS с помощью SSL/TLS
* Включить HSTS и безопасные заголовки
* Ограничить доступ к панелям/админкам
* Блокировать подозрительный трафик (basic DoS/боты)
* Понять основы защиты от атак (XSS, Clickjacking)

---

## 🔐 1. HTTPS и SSL-сертификаты

Самый базовый способ защитить соединение — включить HTTPS.

### Сгенерировать самоподписанный сертификат:

```bash
openssl req -x509 -nodes -days 365 \
  -newkey rsa:2048 \
  -keyout /etc/nginx/ssl/nginx.key \
  -out /etc/nginx/ssl/nginx.crt
```

### Простая настройка HTTPS:

```nginx
server {
    listen 443 ssl;
    server_name example.com;

    ssl_certificate     /etc/nginx/ssl/nginx.crt;
    ssl_certificate_key /etc/nginx/ssl/nginx.key;

    location / {
        proxy_pass http://localhost:5000;
    }
}
```

> 💡 Лучше использовать Let's Encrypt + [certbot](https://certbot.eff.org/).

---

## 📦 2. HSTS (HTTP Strict Transport Security)

Позволяет браузеру **никогда** не использовать HTTP:

```nginx
add_header Strict-Transport-Security "max-age=63072000; includeSubDomains" always;
```

> ⚠️ Только после того, как вы уверены, что HTTPS работает стабильно.

---

## 🧱 3. Безопасные заголовки

```nginx
add_header X-Frame-Options "SAMEORIGIN" always;        # защита от clickjacking
add_header X-XSS-Protection "1; mode=block" always;    # защита от XSS
add_header X-Content-Type-Options "nosniff" always;    # защита от подмены MIME
add_header Referrer-Policy "no-referrer-when-downgrade" always;
add_header Permissions-Policy "geolocation=()" always; # ограничение API браузера
```

---

## 🚫 4. Блокировка опасных/подозрительных запросов

### Простейший способ — через регулярки:

```nginx
location ~* "(eval\(|base64_decode\()" {
    return 403;
}
```

Или блокировать сканеры:

```nginx
if ($http_user_agent ~* (nikto|sqlmap|fuzzer)) {
    return 403;
}
```

---

## 🔐 5. Basic Auth (ограничение доступа по паролю)

### Шаг 1. Установить утилиту:

```bash
sudo apt install apache2-utils
```

### Шаг 2. Создать пользователя:

```bash
htpasswd -c /etc/nginx/.htpasswd admin
```

### Шаг 3. Добавить в конфигурацию:

```nginx
location /admin {
    auth_basic "Restricted Area";
    auth_basic_user_file /etc/nginx/.htpasswd;
}
```

---

## 🚷 6. Ограничения по IP

```nginx
location /admin {
    allow 192.168.1.0/24;
    deny all;
}
```

---

## ⚙️ Пример безопасной конфигурации

```nginx
server {
    listen 443 ssl;
    server_name secure.example.com;

    ssl_certificate     /etc/ssl/certs/example.crt;
    ssl_certificate_key /etc/ssl/private/example.key;

    add_header Strict-Transport-Security "max-age=31536000; includeSubDomains" always;
    add_header X-Frame-Options "SAMEORIGIN" always;
    add_header X-Content-Type-Options "nosniff" always;

    location /admin {
        auth_basic "Admin Area";
        auth_basic_user_file /etc/nginx/.htpasswd;
        allow 192.168.1.0/24;
        deny all;
    }

    location / {
        proxy_pass http://localhost:3000;
    }
}
```

---

## 🧪 Задания для самостоятельной практики

1. Настрой HTTPS (можно через Let's Encrypt).
2. Добавь защитные заголовки.
3. Ограничь доступ к `/admin` по IP и/или паролю.
4. Заблокируй подозрительные `User-Agent`.
5. Проверь, работает ли HSTS через [https://hstspreload.org](https://hstspreload.org).

---

## 📌 Вопросы для самопроверки

* Чем отличается `ssl_certificate` от `ssl_certificate_key`?
* Что делает `X-Frame-Options`?
* Как настроить basic auth в Nginx?
* Зачем нужен HSTS?
* Как заблокировать доступ по User-Agent?

---

Если всё понятно — можем переходить к **Модулю 10: Кэширование и ускорение через Nginx**. Продолжим?


Отлично! Переходим к **Модулю 10: Кэширование и ускорение через Nginx** — одному из ключевых инструментов повышения производительности ваших веб-приложений.

---

# ⚡ Модуль 10: Кэширование и ускорение через Nginx

---

## 🎯 Цели модуля:

* Понять, зачем нужно кэширование и какие типы бывают
* Настроить кэширование статики (CSS, JS, изображения)
* Настроить proxy-кэш для backend-сервисов (например, API)
* Использовать microcaching для высоконагруженных систем
* Управлять кэшированием через заголовки и параметры

---

## 📁 1. Кэширование статических файлов

Самое простое и эффективное: отдавать CSS/JS/изображения прямо из Nginx.

```nginx
location ~* \.(jpg|jpeg|png|gif|ico|css|js)$ {
    expires 30d;
    add_header Cache-Control "public";
}
```

### Что это делает:

* `expires 30d`: браузер кэширует на 30 дней
* `Cache-Control "public"`: можно кэшировать на прокси-серверах

> ⚠️ Не забывай про версии файлов (`style.v2.css`), иначе изменения не подгрузятся.

---

## 🔁 2. Proxy-кэш (для backend-сервисов)

Позволяет сохранить ответ backend-сервера и отдавать его при повторных запросах.

```nginx
proxy_cache_path /var/cache/nginx levels=1:2 keys_zone=my_cache:10m max_size=100m inactive=60m use_temp_path=off;

server {
    location /api/ {
        proxy_pass http://backend:5000;
        proxy_cache my_cache;
        proxy_cache_valid 200 10m;
        proxy_cache_use_stale error timeout updating;
    }
}
```

### Объяснение:

* `proxy_cache_path`: создаёт кэш
* `proxy_cache my_cache`: включает его
* `proxy_cache_valid 200 10m`: кэшировать успешные ответы на 10 минут
* `use_stale`: отдавать старый кэш при ошибках

---

## ⚡ 3. Microcaching

Позволяет кэшировать даже на **1-2 секунды**, что существенно разгружает backend при всплесках трафика.

```nginx
location /news/ {
    proxy_pass http://news-api;
    proxy_cache my_cache;
    proxy_cache_valid 200 1s;
}
```

> Подходит для быстро обновляемых, но часто запрашиваемых ресурсов.

---

## ⚙️ 4. Управление кэшем через заголовки

Можно отдавать или не отдавать кэш в зависимости от заголовков клиента или ответа сервера.

```nginx
proxy_cache_bypass $http_cache_control;
proxy_no_cache $http_cache_control;
```

Например, `Cache-Control: no-cache` в запросе — отключает кэширование.

---

## ❌ 5. Очистка кэша

Кэш можно чистить вручную, удаляя файлы:

```bash
rm -rf /var/cache/nginx/*
```

Или использовать внешние инструменты, например:

* `nginx-cache-purge` (3rd party модуль)
* `PURGE` метод (через специальную настройку)

---

## 🧪 Пример конфигурации proxy cache

```nginx
proxy_cache_path /data/nginx/cache levels=1:2 keys_zone=backend_cache:20m max_size=500m inactive=10m;

server {
    location /api/ {
        proxy_pass http://localhost:8000;
        proxy_cache backend_cache;
        proxy_cache_valid 200 5m;
        proxy_cache_use_stale error timeout updating;
        add_header X-Cache-Status $upstream_cache_status;
    }
}
```

> `X-Cache-Status` помогает видеть статус: `HIT`, `MISS`, `BYPASS`.

---

## 🧪 Практика (опционально)

1. Настрой кэш для изображений и JS/CSS.
2. Включи proxy cache для backend и проверь `X-Cache-Status`.
3. Реализуй microcaching на `/api/top-news`.
4. Протестируй поведение при `Cache-Control: no-cache`.

---

## 📌 Вопросы для самопроверки

* Зачем нужен proxy-кэш и чем он отличается от кэша браузера?
* Что такое microcaching и когда его стоит использовать?
* Какой параметр отвечает за хранение кэша в памяти?
* Что делает `proxy_cache_use_stale`?

---

Готов двигаться дальше? Следующий — **Модуль 11: Nginx как балансировщик нагрузки**.


Конечно! Переходим к **Модулю 11: Nginx как балансировщик нагрузки (расширенно)** — важнейшему инструменту масштабирования и высокой доступности систем.

---

# ⚖️ Модуль 11: Nginx как балансировщик нагрузки

---

## 🎯 Цели модуля:

* Понять виды балансировки нагрузки в Nginx
* Настроить простую и продвинутую балансировку
* Изучить sticky-сессии, failover, health-check
* Использовать балансировку с HTTPS и API
* Сравнить с другими решениями (HAProxy, Traefik)

---

## 📚 1. Виды балансировки в Nginx

В Nginx есть несколько встроенных стратегий:

| Метод         | Описание                                |
| ------------- | --------------------------------------- |
| `round-robin` | По очереди (по умолчанию)               |
| `least_conn`  | Наименьшее количество соединений        |
| `ip_hash`     | Привязка пользователя к одному серверу  |
| `random`      | Случайный выбор (в коммерческой версии) |

---

## ⚙️ 2. Простая балансировка: `round-robin`

```nginx
upstream backend {
    server backend1.example.com;
    server backend2.example.com;
}

server {
    location / {
        proxy_pass http://backend;
    }
}
```

> Запросы пойдут поочерёдно на `backend1` и `backend2`.

---

## ⚙️ 3. Балансировка с `least_conn`

```nginx
upstream backend {
    least_conn;
    server backend1.example.com;
    server backend2.example.com;
}
```

> Используется, когда время обработки запроса сильно варьируется (например, в API).

---

## 🔄 4. Sticky-сессии с `ip_hash`

```nginx
upstream backend {
    ip_hash;
    server backend1.example.com;
    server backend2.example.com;
}
```

> Один и тот же пользователь по IP всегда попадает на один и тот же backend. Особенно важно при сессионной авторизации.

---

## ❌ 5. Отказоустойчивость и failover

```nginx
upstream backend {
    server backend1.example.com;
    server backend2.example.com backup;
}
```

* `backup` — используется только если основной сервер недоступен.
* Также можно указать параметры:

```nginx
server backend1.example.com max_fails=3 fail_timeout=30s;
```

---

## ❤️ 6. Проверка живости (health checks)

В **бесплатной версии Nginx** встроенной активной проверки живости нет. Есть только пассивная через `max_fails`.

> Для полноценного health-check используют:

* [nginx plus (коммерческий)](https://www.nginx.com/products/)
* Внешние инструменты (например, keepalived, consul)
* Lua-модули и OpenResty

---

## 🔒 7. Балансировка с HTTPS

```nginx
upstream backend {
    server backend1.example.com:443;
    server backend2.example.com:443;
}

server {
    listen 443 ssl;
    ssl_certificate     /etc/nginx/cert.pem;
    ssl_certificate_key /etc/nginx/key.pem;

    location / {
        proxy_pass https://backend;
        proxy_ssl_verify off;
    }
}
```

> Проксируем на **HTTPS-бэкэнды**, отключаем верификацию сертификатов (можно настроить иначе).

---

## 🧪 8. Отладка и наблюдение

Добавь заголовок, чтобы видеть, какой backend сработал:

```nginx
proxy_set_header X-Upstream-Host $upstream_addr;
```

> Используй логи или curl-запросы, чтобы отслеживать, как распределяются запросы.

---

## 🧪 Пример: балансировка API с sticky и failover

```nginx
upstream api_backend {
    ip_hash;
    server api1.local:8000 max_fails=3 fail_timeout=10s;
    server api2.local:8000 backup;
}

server {
    location /api/ {
        proxy_pass http://api_backend;
        proxy_set_header X-Upstream-Host $upstream_addr;
    }
}
```

---

## 📌 Вопросы для самопроверки

* Какие виды балансировки есть в Nginx?
* Чем `ip_hash` отличается от `least_conn`?
* Как реализовать отказоустойчивость?
* Что делать, если нужен активный health-check в Open Source версии?

---

Готов двигаться дальше? В **финальном модуле** мы разберём **мониторинг, логирование и отладку Nginx**, чтобы выявлять узкие места и реагировать на инциденты.


Отлично, переходим к **Модулю 12: Мониторинг, логирование, отладка и best practices в Nginx**.

---

# 🧩 Модуль 12: Мониторинг, логирование, отладка и best practices

---

## 🎯 Цели модуля:

* Научиться настраивать access и error логи
* Анализировать логи вручную и через инструменты
* Понять, какие метрики важны для мониторинга
* Освоить отладку проблем с конфигурацией
* Собрать набор best practices для продакшна

---

## 📄 1. Access log и error log

**Логи по умолчанию** находятся здесь:

```nginx
access_log /var/log/nginx/access.log;
error_log  /var/log/nginx/error.log warn;
```

### 💡 Пример access-лога:

```
127.0.0.1 - - [01/Aug/2025:13:00:00 +0300] "GET /api HTTP/1.1" 200 512 "-" "curl/8.0"
```

### ✍️ Пользовательский формат:

```nginx
log_format custom '$remote_addr - $remote_user [$time_local] '
                  '"$request" $status $body_bytes_sent '
                  '"$http_referer" "$http_user_agent" '
                  '$request_time $upstream_response_time';

access_log /var/log/nginx/access.log custom;
```

---

## 🔍 2. Анализ логов

### 📊 Инструменты:

* `goaccess` — интерактивная CLI-статистика
* `awk`, `grep`, `cut`, `sort` — ручной анализ
* `logrotate` — ротация логов
* Интеграция с **Elasticsearch + Kibana (ELK stack)**

---

## 🩺 3. Важные метрики и мониторинг

Хотя Nginx не экспортирует метрики сам по себе, можно:

### 📡 Использовать stub\_status

```nginx
location /nginx_status {
    stub_status;
    allow 127.0.0.1;
    deny all;
}
```

Пример вывода:

```
Active connections: 3
server accepts handled requests
 10 10 20
Reading: 0 Writing: 1 Waiting: 2
```

### 🛠 Prometheus + nginx-exporter:

* Установи [nginx-prometheus-exporter](https://github.com/nginxinc/nginx-prometheus-exporter)
* Экспортируй метрики в Prometheus и визуализируй через Grafana

---

## 🐞 4. Отладка ошибок

* `nginx -t` — проверка конфигурации
* `nginx -s reload` — безопасная перезагрузка
* `tail -f /var/log/nginx/error.log` — реальный вывод ошибок
* `curl -I http://localhost` — проверь заголовки и код ответа
* Включи **debug-режим**:

```nginx
error_log /var/log/nginx/error.log debug;
```

---

## ✅ 5. Best Practices для продакшна

### 📋 Безопасность

* Не раскрывай версию: `server_tokens off`
* Используй `include /etc/nginx/snippets/ssl-params.conf;` для безопасных TLS-настроек
* Ограничивай доступ к конфигурации через `allow/deny`

### ⚙️ Производительность

* Используй `worker_processes auto;` и `worker_connections` на основе нагрузки
* Кэшируй статичные ресурсы
* Компрессируй с помощью `gzip` или `brotli`

### 📦 Организация

* Разделяй конфигурации на файлы: `sites-available`, `snippets`, `conf.d`
* Комментируй и документируй каждую секцию
* Используй CI/CD для проверки и развёртывания (`nginx -t` в пайплайне)

---

## 🧠 Вопросы для самопроверки

* Как посмотреть активные соединения к Nginx?
* Как изменить формат логов?
* Как проверить конфигурацию на ошибки?
* Какие меры безопасности следует предпринять?

---

## ✅ Что дальше?

Ты завершил 12 модулей курса! 🙌
Осталось закрепить знания через **финальную лабораторную работу**.

Хочешь, чтобы я её составил?


Конечно! Вот шпаргалка по **Nginx** — кратко и по делу:

---

# 📄 Шпаргалка по NGINX

---

## 📦 Установка (Ubuntu)

```bash
sudo apt update
sudo apt install nginx
sudo systemctl start nginx
sudo systemctl enable nginx
```

---

## ⚙️ Проверка конфигурации

```bash
nginx -t         # Проверка конфигурации
nginx -s reload  # Перезапуск с обновлением конфигурации
```

---

## 📁 Структура каталогов (по умолчанию в Debian/Ubuntu)

```
/etc/nginx/
├── nginx.conf               # Главный конфигурационный файл
├── conf.d/                  # Доп. конфиги
├── sites-available/        # Доступные сайты
├── sites-enabled/          # Активные сайты (симлинки)
├── snippets/               # Вставки конфигураций
```

---

## 🖥 Базовый конфиг сайта (Virtual Host)

```nginx
server {
    listen 80;
    server_name example.com;

    root /var/www/example;
    index index.html index.htm;

    location / {
        try_files $uri $uri/ =404;
    }
}
```

---

## 🔁 Reverse Proxy

```nginx
location /api/ {
    proxy_pass http://127.0.0.1:8000/;
    proxy_set_header Host $host;
    proxy_set_header X-Real-IP $remote_addr;
}
```

---

## 🔐 SSL (Let's Encrypt + Certbot)

```bash
sudo apt install certbot python3-certbot-nginx
sudo certbot --nginx -d example.com -d www.example.com
```

---

## 📊 Логи

```nginx
access_log /var/log/nginx/access.log;
error_log  /var/log/nginx/error.log;
```

---

## 📡 Мониторинг (stub\_status)

```nginx
location /nginx_status {
    stub_status;
    allow 127.0.0.1;
    deny all;
}
```

---

## 🧩 Балансировка нагрузки

```nginx
upstream backend {
    server 127.0.0.1:8000;
    server 127.0.0.1:8001;
}

server {
    location / {
        proxy_pass http://backend;
    }
}
```

---

## 📥 Кэширование

```nginx
location ~* \.(jpg|jpeg|png|gif|ico|css|js)$ {
    expires 30d;
    access_log off;
}
```

---

## 🔁 Redirect / Rewrite

```nginx
return 301 https://$host$request_uri;

rewrite ^/old-page$ /new-page permanent;
```

---

## 🔐 Защита

```nginx
# Ограничить доступ
location /admin {
    allow 192.168.1.0/24;
    deny all;
}

# Базовая авторизация
location /secure {
    auth_basic "Restricted Area";
    auth_basic_user_file /etc/nginx/.htpasswd;
}
```

---

## 💨 Gzip-сжатие

```nginx
gzip on;
gzip_types text/plain application/json text/css application/javascript;
```

---

## 🧠 Часто используемые переменные

| Переменная      | Описание                       |
| --------------- | ------------------------------ |
| `$host`         | Имя хоста из запроса           |
| `$remote_addr`  | IP-адрес клиента               |
| `$request_uri`  | Полный URI                     |
| `$uri`          | URI без аргументов             |
| `$args`         | Аргументы запроса (?key=value) |
| `$request_time` | Время обработки запроса (сек)  |

---

Если нужно — могу подготовить шпаргалку в PDF или с визуальной схемой. Скажешь?
